https://livy.incubator.apache.org/examples/

In [1]:
import json, pprint, requests, textwrap

In [2]:
host = 'http://spark-livy:8998'
data = {'kind': 'pyspark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 5,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [3]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 5,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['22/04/03 05:28:49 INFO StandaloneAppClient$ClientEndpoint: Connecting to master spark://master:7077...',
  '22/04/03 05:28:49 INFO TransportClientFactory: Successfully created connection to master/172.18.0.21:7077 after 205 ms (0 ms spent in bootstraps)',
  '22/04/03 05:28:50 INFO StandaloneSchedulerBackend: Connected to Spark cluster with app ID app-20220403052850-0008',
  "22/04/03 05:28:50 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 34022.",
  '22/04/03 05:28:50 INFO NettyBlockTransferService: Server created on localhost:34022',
  '22/04/03 05:28:50 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy',
  '22/04/03 05:28:50 INFO BlockManagerMaster: Registering BlockManager BlockManage

In [4]:
session_url

'http://spark-livy:8998/sessions/5'

In [6]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'code': '1 + 1',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [7]:
r.headers

{'Date': 'Sun, 03 Apr 2022 05:29:58 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Location': '/sessions/5/statements/0', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}

In [8]:
statements_url

'http://spark-livy:8998/sessions/5/statements'

In [9]:
r = requests.get(statements_url, headers=headers)
pprint.pprint(r.json())

{'statements': [{'code': '1 + 1',
                 'completed': 1648963800614,
                 'id': 0,
                 'output': {'data': {'text/plain': '2'},
                            'execution_count': 0,
                            'status': 'ok'},
                 'progress': 1.0,
                 'started': 1648963800588,
                 'state': 'available'}],
 'total_statements': 1}


In [10]:
data = {
  'code': textwrap.dedent("""

    # Create RDD from parallelize    
    dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
    rdd=spark.sparkContext.parallelize(dataList)
    """)
}

In [11]:
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         '\n'
         '# Create RDD from parallelize    \n'
         'dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]\n'
         'rdd=spark.sparkContext.parallelize(dataList)\n',
 'completed': 0,
 'id': 1,
 'output': None,
 'progress': 0.0,
 'started': 1648963881766,
 'state': 'running'}


In [12]:
r = requests.get(statements_url, headers=headers)
pprint.pprint(r.json())

{'statements': [{'code': '1 + 1',
                 'completed': 1648963800614,
                 'id': 0,
                 'output': {'data': {'text/plain': '2'},
                            'execution_count': 0,
                            'status': 'ok'},
                 'progress': 1.0,
                 'started': 1648963800588,
                 'state': 'available'},
                {'code': '\n'
                         '\n'
                         '# Create RDD from parallelize    \n'
                         'dataList = [("Java", 20000), ("Python", 100000), '
                         '("Scala", 3000)]\n'
                         'rdd=spark.sparkContext.parallelize(dataList)\n',
                 'completed': 1648963882618,
                 'id': 1,
                 'output': {'data': {'text/plain': ''},
                            'execution_count': 1,
                            'status': 'ok'},
                 'progress': 1.0,
                 'started': 1648963881766,
      

In [13]:
requests.delete(session_url, headers=headers)

<Response [200]>

In [12]:
session_url = 'http://spark-livy:8998/sessions/4'